In [25]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup
import csv
import re
import itertools as iter
import copy

In [2]:
def download_files():
    """Downloads all index files from EDGAR into the current directory"""
    
    for year in range(1993, 2019):
        for quarter in range(1, 5):
            url = 'https://www.sec.gov/Archives/edgar/full-index/' + str(year) + '/QTR' + str(quarter) + '/company.idx'
            r = req.get(url, allow_redirects=True)
            with open('Index-' + str(year) + '-' + str(quarter) + '.txt', "wb") as f:
                f.write(r.content)

In [93]:
import os

def get_reports(company_name, startyear, startquarter, endyear, endquarter):
    """
    Reads all index files and returns a generator of URLs of all 10-K/10-Qs of the given company.
    
    Parameters:
    company_name (str): name of the company 
    startyear (int): the year to start looking for reports, inclusive
    startquarter (int): the quarter in startyear to start looking for reports, inclusive
    endyear (int): the year to stop looking for reports, inclusive
    endquarter(int): the quarter in endyear to stop looking for reports, inclusive
    
    Returns:
    Generator: each element is a tuple with the form (year of report, quarter of report, url of report)
    
    """
    
    files = filter(lambda x: x[-4:] == '.txt', os.listdir('.'))
    for file in files:
        year = int(file[6:10])
        if year < startyear or year > endyear:
            continue
        quarter = int(file[11])
        if (year == startyear and quarter < startquarter) or (year == endyear and quarter > endquarter):
            continue
        txt = open(file, 'r')
        for line in txt:
            if line.lower().startswith(company_name.lower()) and '10-Q ' in line: 
                url_index = line.index('edgar/')
                url = line[url_index:].rstrip('\n').strip()
                yield (year, quarter, 'https://www.sec.gov/Archives/' + url)

In [117]:
def make_tables(company_name, text, limit, search_terms, year, quarter, mapping, default_side):
    """
    Parses HTML text and returns a DataFrame containing the desired data
    
    Parameters:
    company_name (str): the name of the company
    text (str): the text of the company report
    limit (int): the max number of tables to search through in the report
    search_terms (list of str): the terms to filter each table for. 
    year (int): the year the document corresponds to
    quarter(int): the quarter the document corresponds to
    mapping (dict): a dict with items of the form (term: [locations, amount])
        term (str): one of the search terms for which detailed location/amount information is required
        locations (list of int): location(s) of the targeted value in the row
        amount (int): the number of times to read in values of this term
    default_side (str): the default location of the number to keep on each row (left/right) if the label is not in mapping
    
    Returns:
    DataFrame: a table containing values in the document corresponding to each of the search terms
    
    """
    regex = re.compile('[^a-z A-Z]')
    soup = BeautifulSoup(text)
    tables = soup.find_all("table")
    column_list = ['Year', 'Quarter']
    data_values = [year, quarter]
    found_terms = [] # By default, only the first occurence of a term in a row label will be used
    table_num = 0
    term_map = copy.deepcopy(mapping)
    
    for table in tables:
        added_rows = False
        for row in table.find_all('tr'):
            data_row = []
            columns = row.find_all('td')
            row_label = regex.sub('', columns[0].get_text()).strip()
            for term in search_terms:
                if term in row_label:
                    if term in found_terms and term not in term_map:
                        break
                    row_values = []
                    for i in range(1, len(columns)):
                        cell_text = columns[i].get_text()
                        if any(char.isdigit() for char in cell_text):
                            for punctuation in [',', '(', ')', '\n']:
                                cell_text = cell_text.replace(punctuation, '')
                            row_values.append(cell_text)
                    if len(row_values) == 0: 
                        break
                    found_terms.append(term)
                    if term in term_map:
                        term_amount = term_map[term][1]
                        if term_amount == 0:
                            break
                        column_label = term
                        term_start_amount = mapping[term][1]
                        if term_start_amount > 1:
                            column_label += str(term_start_amount - term_amount + 1)
                        positions = term_map[term][0]
                        for position in positions:
                            if len(positions) == 1:
                                column_list.append(column_label)
                            else:
                                column_list.append(column_label + ' [' + str(position) + ']')
                            data_values.append(float(row_values[position]))
                        term_map[term][1] -= 1
                    else:
                        column_list.append(term)
                        if default_side == 'left':
                            data_values.append(float(row_values[0]))
                        else:
                            data_values.append(float(row_values[-1]))
                    added_rows = True
                    break
        
        if added_rows:
            table_num += 1
        if table_num >= limit:
            break
    #print(data_values)
    #print(column_list)
    return pd.DataFrame([data_values], index=[company_name + ' ' + str(year) + '-' + str(quarter)], columns=column_list)

In [95]:
def get_html_tables(company_name, search_terms, startyear, startquarter=1, endyear=2018, endquarter=4, 
                    limit=3, mapping=dict(), default_side='left'):
    """
    Forms tables of the desired search terms of the given company in the given timeframe
    
    Parameters:
    See documentation of get_reports and make_tables.
    
    Returns:
    Generator: each element is a tuple with the form (year, quarter, amalgamated table)
    
    """
    
    reports = get_reports(company_name, startyear, startquarter, endyear, endquarter)
    for report in reports:
        year = report[0]
        quarter = report[1]
        text = req.get(report[2]).text
        if '<html' in text or '<HTML' in text:
            #print(report[0], 'Q', report[1])
            data = make_tables(company_name, text, limit, search_terms, year, quarter, mapping, default_side)
            yield (year, quarter, data)

In [137]:
other_companies = ['Google/Alphabet Inc', 'Intel Inc', 'International Business Machines', 'HP Inc', 'Dell Inc', 
                   'Cisco Systems Inc', 'Advanced Micro Devices Inc', 'Texas Instruments', 'Nvidia']


In [136]:
def get_google_data():
    searchterms_google = ['Revenues', 'Google advertising and other', 'Cost of revenues', 'Total costs and expenses', 
                           'Net income', 'Diluted', 'diluted', 'Total current assets', 'Total assets', 
                           'Total current liabilities', 'Total stockholders equity', 'Depreciation', 
                           'Net cash provided by operating activities', 'Cash and cash equivalents at beginning', 
                          'Cash and cash equivalents at end']
    mappings_google = {'Revenues': [[1], 1], 'Google advertising and other': [[1], 1], 'Diluted': [[1], 1],
                       'diluted': [[1], 1], 'Cost of revenues': [[1], 1], 'Total costs and expenses':[[1], 1], 
                       'Net income':[[1], 1]}
    reports_google = get_html_tables('Google Inc', startyear=2008, startquarter=1, endyear=2015, endquarter=4, 
                                      mapping=mappings_google, search_terms=searchterms_google, default_side='right')
    reports_alphabet = get_html_tables('Alphabet Inc', startyear=2016, startquarter=1, endyear=2018, endquarter=4, 
                                      mapping=mappings_google, search_terms=searchterms_google, default_side='right')
    reports_google_combined = iter.chain(reports_google, reports_alphabet)
    
    combined_df = pd.DataFrame()
        
    for report in reports_google_combined:
        year = report[0]
        quarter = report[1]
        df = report[2].copy()
        #display(df)
        if 'Google advertising and other' in df.columns:
            new_columns = df.columns.tolist()
            new_columns[new_columns.index('Google advertising and other')] = 'Revenues'
            df.columns = new_columns
        if 'diluted' in df.columns:
            if 'Diluted' in df.columns:
                df.drop('diluted', axis=1, inplace=True)
            else:
                new_columns = df.columns.tolist()
                new_columns[new_columns.index('diluted')] = 'Diluted'
                df.columns = new_columns
        if year < 2010: # After 2010, Google changed from counting in thousands to counting in millions
            for col in df.columns:
                if col.lower() != 'diluted' and col.lower() != 'year' and col.lower() != 'quarter':
                    df[col] = round(df[col] / 1000)
        combined_df = combined_df.append(df)
    
    new_columns = ['Earnings/diluted share' if col == 'Diluted' else col for col in combined_df.columns.tolist()]
    combined_df.columns = new_columns
    return combined_df

In [138]:
def get_apple_data():    
    searchterms_apple = ['Net sales', 'Cost of sales', 'Total operating expenses', 'Net income', 'Diluted', 
                         'Total current assets', 'Total assets', 'Total current liabilities', 'Total liabilities', 
                         'Total shareholders equity', 'Cash and cash equivalents beginning of the period', 
                         'Depreciation', 'Cash generated by operating activities', 
                         'Cash and cash equivalents end of the period']
    mappings_apple = {'Diluted':[[0], 2]}
    reports_apple = get_html_tables('Apple Inc', startyear=2008, startquarter=1, endyear=2018, endquarter=4, 
                                    mapping=mappings_apple, search_terms=searchterms_apple)

    combined_df = pd.DataFrame()

    for report in reports_apple:
        year = report[0]
        quarter = report[1]
        df = report[2].copy()
        #display(df)
        #print(year, quarter)
        combined_df = combined_df.append(df)
    
    new_columns = ['Earnings/diluted share' if col == 'Diluted1' 
                   else 'Number of diluted shares (thousands)' if col == 'Diluted2'
                   else col for col in combined_df.columns.tolist()]
    combined_df.columns = new_columns
    return combined_df